In [ ]:
import json
import pandas as pd
import chardet
import re

: 

In [ ]:
# Leer los primeros bytes del archivo para detectar la codificación
with open('tioberny.txt', 'rb') as file:
    raw_data = file.read()

# Detectar la codificación
result = chardet.detect(raw_data)
encoding = result['encoding']

# Lista para almacenar todos los objetos JSON
data_list = []

# Leer el archivo línea por línea
with open('tioberny.txt', 'r', encoding=encoding) as file:
    for line in file:
        try:
            # Intentar convertir cada línea en un objeto JSON
            json_data = json.loads(line)
            data_list.append(json_data)
        except json.JSONDecodeError as e:
            print(f"Error en la línea: {line}, error: {e}")

# Crear un DataFrame con todos los objetos JSON
df = pd.json_normalize(data_list)

# Mostrar el DataFrame
df


# 3 Preprocesamiento de Datos

#### Convertir texto a minúsculas

In [ ]:
df['rawContent'] = df['rawContent'].str.lower()


#### Eliminar caracteres especiales como “@”, “#”, urls, emoticonos:

In [ ]:

# Eliminar URLs
df['rawContent'] = df['rawContent'].apply(lambda x: re.sub(r'http\S+', '', x))

# Eliminar menciones, hashtags y emoticonos
df['rawContent'] = df['rawContent'].apply(lambda x: re.sub(r'[@#]\w+', '', x))

# Eliminar emoticonos unicode
df['rawContent'] = df['rawContent'].str.replace(r'[^\x00-\x7F]+', '', regex=True)

# Eliminar signos de puntuación
df['rawContent'] = df['rawContent'].apply(lambda x: re.sub(r'[^\w\s]', '', x))


#### Eliminar stopwords y números

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('spanish'))
df['rawContent'] = df['rawContent'].apply(lambda x: ' '.join([word for word in x.split() if word not in stop_words]))


#### Eliminar duplicados

In [ ]:
df.drop_duplicates(subset=['id'], inplace=True)


#### Extraccion de interacciones

In [ ]:
df['mentions'] = df['rawContent'].apply(lambda x: re.findall(r'@\w+', x))


In [ ]:
df['is_retweet'] = df['rawContent'].apply(lambda x: 1 if 'rt @' in x.lower() else 0)


# 4 Analisis Explotartorio

#### Número de tweets, usuarios únicos, menciones

In [ ]:
total_tweets = len(df)
unique_users = df['user.id'].nunique()
total_mentions = df['mentions'].apply(len).sum()

print(f"Total de tweets: {total_tweets}")
print(f"Usuarios únicos: {unique_users}")
print(f"Total de menciones: {total_mentions}")


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

text = ' '.join(df['rawContent'])
wordcloud = WordCloud(width=800, height=400).generate(text)

plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()


# 5 Análisis de la Topología de la Red

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Crear un grafo dirigido
G = nx.DiGraph()

# Añadir nodos y aristas para las menciones
for idx, row in df.iterrows():
    user = row['user.id_str']
    mentions = row['mentions']
    for mention in mentions:
        G.add_edge(user, mention)

# Visualización del grafo con layout mejorado
plt.figure(figsize=(12, 12))

# Utilizar spring_layout para mejorar la distribución de los nodos
pos = nx.spring_layout(G, k=0.5, iterations=50)

# Dibujar el grafo con opciones adicionales para personalización
nx.draw(G, pos, with_labels=True, 
        node_size=700,        # Tamaño de los nodos
        node_color="skyblue", # Color de los nodos
        font_size=10,         # Tamaño de la fuente de las etiquetas
        font_color="black",   # Color del texto
        edge_color="gray",    # Color de las aristas
        linewidths=0.5,       # Grosor del borde de los nodos
        arrows=True,          # Mostrar flechas para indicar la dirección de las aristas
        arrowsize=15          # Tamaño de las flechas
       )

# Mostrar el grafo
plt.show()
